In [235]:
import pandas as pd
from pathlib import Path
import os
from statistics import mode
import numpy as np

## Decompose: final label -> individual

In [256]:
all_decompose = {}
label = 0

# All combined
for m in range(3):
    for g in range(2):
        for a in range(3):
            all_decompose[label] = ':'.join([str(m),str(g),str(a)])
            label += 1
            
# mask & gender

mask_gender_decompose = {}
label = 0

for m in range(3):
    for g in range(2):
            mask_gender_decompose[label] = ':'.join([str(m),str(g)])
            label += 1

# Age 
age_decompose = {
    0: 0,
    1: 0,
    2: 1,
    3: 1,
    4: 1,
    5: 2
}

## individual -> final label

In [257]:
# All combined (mask, gender, age)
label = 0
label_trsfm = {}

for m in range(3):
    for g in range(2):
        for a in range(3):
            label_trsfm[':'.join([str(m),str(g),str(a)])] = label
            label += 1

## csv directory

In [258]:
all_csv_dir = Path('./out/all_hardvoting')
mask_gender_csv_dir = Path('./out/mask_gender_hardvoting')
age_csv_dir = Path('./out/age_hardvoting')

In [259]:
all_csv_names = os.listdir(all_csv_dir)
mask_gender_csv_names = os.listdir(mask_gender_csv_dir)
age_csv_names = os.listdir(age_csv_dir)

In [260]:
all_csv_names

['resnet_dropout_0.7_2.csv',
 'CJ_submission_74_4.csv',
 'efficientnet.csv',
 'efficientnetb2_b2.csv',
 '.ipynb_checkpoints',
 'RES18_SEG_CROP(DH).csv',
 'densenet.csv']

In [261]:
all_df_list = []
mask_gender_df_list = []
age_df_list = []

In [262]:
DH_age_csv = pd.read_csv('./out/DH_age.csv')

In [263]:
for csv in all_csv_names:
    if not csv.endswith('.csv'):
        continue
    all_df_list.append(pd.read_csv(all_csv_dir / csv))
    
for csv in mask_gender_csv_names:
    if not csv.endswith('.csv'):
        continue
    mask_gender_df_list.append(pd.read_csv(mask_gender_csv_dir / csv))
    
for csv in age_csv_names:
    if not csv.endswith('.csv'):
        continue
    age_df_list.append(pd.read_csv(age_csv_dir / csv))

In [297]:
submitted_csv = pd.read_csv('hard_voting_submission1.csv')

In [298]:
submitted_csv

,Unnamed: 0,ImageID,ans
0,0,cbc5c6e168e63498590db46022617123f1fe1268.jpg,13
1,1,0e72482bf56b3581c081f7da2a6180b8792c7089.jpg,1
2,2,b549040c49190cedc41327748aeb197c1670f14d.jpg,13
3,3,4f9cb2a045c6d5b9e50ad3459ea7b791eb6e18bc.jpg,13
4,4,248428d9a4a5b6229a7081c32851b90cb8d38d0c.jpg,12
...,...,...,...
12595,12595,d71d4570505d6af8f777690e63edfa8d85ea4476.jpg,1
12596,12596,6cf1300e8e218716728d5820c0bab553306c2cfd.jpg,4
12597,12597,8140edbba31c3a824e817e6d5fb95343199e2387.jpg,9
12598,12598,030d439efe6fb5a7bafda45a393fc19f2bf57f54.jpg,1


In [378]:
final_label = []
for idx in range(12600):
    freq = {
        'mask': [],
        'gender': [],
        'age': []
    }
    
    # For All (mask, gender, age)
    for df_idx in range(len(all_df_list)):
        combined_label = all_df_list[df_idx].loc[idx, 'ans']
        mask, gender, age = all_decompose[combined_label].split(':')
        freq['mask'].append(int(mask))
        freq['gender'].append(int(gender))
        freq['age'].append(int(age))
    
    # For Mask & Gender
    for df_idx in range(len(mask_gender_df_list)):
        combined_label = mask_gender_df_list[df_idx].loc[idx, 'ans']
        mask, gender = mask_gender_decompose[combined_label].split(':')
        freq['mask'].append(int(mask))
        freq['gender'].append(int(gender))
        
#     For Age
    for df_idx in range(len(age_df_list)):
        age6 = age_df_list[df_idx].loc[idx, 'age']
        age3 = age_decompose[age6]
#         freq['age'].append(int(age3))

#     print("submitted:",all_decompose[submitted_csv.loc[idx, 'ans']].split(':')[2])
#     freq['age'].append(int(all_decompose[submitted_csv.loc[idx, 'ans']].split(':')[2]))
    # DH age
    freq['age'].append(int(DH_age_csv.loc[idx, 'ans']))
#     print("DH:",DH_age_csv.loc[idx, 'ans'])
    
    
    mask_mode = str(mode(freq['mask']))
    gender_mode = str(mode(freq['gender']))
    age_mode = str(mode(freq['age']))
    final_label.append(label_trsfm[":".join([mask_mode, gender_mode, age_mode])])
    
    # Print final lable for current image
    # print("Final label: {}".format(label_trsfm[":".join([mask_mode, gender_mode, age_mode])]))

In [379]:
sum(np.array(final_ans) != np.array(final_label))

542

In [352]:
np.sum(np.array(list(submitted_csv['ans'])) != np.array(final_label))

521

In [371]:
df1['ans'] = final_label

In [361]:
df1.to_csv('hard_voting_submission6.csv')

In [372]:
df1

,ImageID,ans
0,cbc5c6e168e63498590db46022617123f1fe1268.jpg,13
1,0e72482bf56b3581c081f7da2a6180b8792c7089.jpg,1
2,b549040c49190cedc41327748aeb197c1670f14d.jpg,13
3,4f9cb2a045c6d5b9e50ad3459ea7b791eb6e18bc.jpg,13
4,248428d9a4a5b6229a7081c32851b90cb8d38d0c.jpg,12
...,...,...
12595,d71d4570505d6af8f777690e63edfa8d85ea4476.jpg,1
12596,6cf1300e8e218716728d5820c0bab553306c2cfd.jpg,4
12597,8140edbba31c3a824e817e6d5fb95343199e2387.jpg,9
12598,030d439efe6fb5a7bafda45a393fc19f2bf57f54.jpg,1


In [377]:
df1.to_csv('hard_voting_submission7.csv')

In [375]:
sum(np.array(df1['ans']) != np.array(final_ans))

521

In [376]:
df1

,ImageID,ans
0,cbc5c6e168e63498590db46022617123f1fe1268.jpg,13
1,0e72482bf56b3581c081f7da2a6180b8792c7089.jpg,1
2,b549040c49190cedc41327748aeb197c1670f14d.jpg,13
3,4f9cb2a045c6d5b9e50ad3459ea7b791eb6e18bc.jpg,13
4,248428d9a4a5b6229a7081c32851b90cb8d38d0c.jpg,12
...,...,...
12595,d71d4570505d6af8f777690e63edfa8d85ea4476.jpg,1
12596,6cf1300e8e218716728d5820c0bab553306c2cfd.jpg,4
12597,8140edbba31c3a824e817e6d5fb95343199e2387.jpg,9
12598,030d439efe6fb5a7bafda45a393fc19f2bf57f54.jpg,1
